In [ ]:
from pathlib import Path
import os

# === Configure your settings here ===
# - Set your project root path
# - We build the MCP server config inline using ROOT and API_DOMAIN
# - Optionally paste your OpenAI API key (or leave empty to use environment)

ROOT = Path("/Users/eloh/egain-mcp")  # <- change to your local path
API_DOMAIN = "api.aidev.egain.cloud"         # <- change to your API domain (e.g., api.aidev.egain.cloud)

# OpenAI
OPENAI_API_KEY = ""                    # <- paste here or set env var OPENAI_API_KEY
OPENAI_MODEL = "gpt-4o-mini"           # <- change model if desired

# Apply environment variables for downstream cells
if OPENAI_API_KEY:
    os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

print(f"ROOT set to: {ROOT}")
print("API_DOMAIN:", API_DOMAIN)
print("OPENAI_MODEL:", OPENAI_MODEL)


ROOT set to: /Users/eloh/egain-mcp
API_DOMAIN: api.aidev.egain.cloud
OPENAI_MODEL: gpt-4o-mini


# Build Your Own MCP Client (JSON config driven)

A friendly, step-by-step guide to connect from Jupyter to your Node MCP server using a JSON config (Claude/Cursor-style `mcpServers`). PLEASE PROVIDE A FREE OPENAI TOKEN TO TEST, SEE CODE BELOW.

### What you’ll do
- Install the Python MCP client
- Load server settings from an `.json` MCP config
- Start the server via stdio
- List available tools
- Optionally call a tool

### Works with configs like
```json
{
  "mcpServers": {
    "egain-mcp": {
      "command": "node",
      "args": [
        "/Users/eloh/test-knowledge-mcp-typescript/bin/mcp-server.js",
        "start",
        "--api-domain",
        "api.aidev.egain.cloud"
      ]
    }
  }
}
```

If you use Claude or Cursor, this will auto-detect `~/.cursor/mcp.json` or `~/Library/Application Support/Claude/claude_desktop_config.json`. You can also point to any custom JSON path.


## Step 1: Install requirements

We’ll use the official Python MCP client. Run this once (restart kernel if imports fail):


In [9]:
# Install and import dependencies
%pip install -U "mcp==1.19.0" anyio

import json
import os
from typing import Dict, Any

try:
    from mcp import StdioServerParameters
    from mcp.client.stdio import stdio_client
    from mcp.client.session import ClientSession
    print("✅ MCP imports ready")
except Exception as e:
    print("❌ MCP import failed. Ensure 'mcp' is installed and kernel is restarted if needed.")
    raise



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
✅ MCP imports ready


## Step 2: Build your MCP server config inline

We construct the MCP server settings right here using `ROOT` and `API_DOMAIN` from the first cell. No external JSON files are needed.

Equivalent JSON shape (for reference only):
```json
{
  "mcpServers": {
    "EgainMcp": {
      "command": "node",
      "args": ["/ABSOLUTE/PATH/bin/mcp-server.js", "start", "--api-domain", "api.example.com"]
    }
  }
}
```


In [10]:
# Build MCP server settings inline using ROOT and API_DOMAIN (from the first cell)
from pathlib import Path

ROOT = globals().get("ROOT", Path.cwd().parent)
API_DOMAIN = globals().get("API_DOMAIN", "api.example.com")

server_cfg = {
    "command": "node",
    "args": [
        str(ROOT / "bin" / "mcp-server.js"),
        "start",
        "--api-domain",
        API_DOMAIN,
    ]
}

print("Using inline MCP server config:")
print(json.dumps(server_cfg, indent=2))


Using inline MCP server config:
{
  "command": "node",
  "args": [
    "/Users/eloh/egain-mcp/bin/mcp-server.js",
    "start",
    "--api-domain",
    "api.aidev.egain.cloud"
  ]
}


## Step 3: Connect to the MCP server and list tools

Connect to your MCP server via stdio using the config above, then enumerate the available tools to verify connectivity.


In [11]:
# Connect to the MCP server (stdio) and list tools

async def list_tools_from_config(server: Dict[str, Any]):
    params = StdioServerParameters(
        command=server["command"],
        args=server.get("args", []),
        env=server.get("env", None),
    )
    async with stdio_client(params) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()
            tools_result = await session.list_tools()
            tools = tools_result.tools
            print(f"Found {len(tools)} tools:")
            for t in tools:
                print(f"- {t.name}")
            return tools

# Run it
await list_tools_from_config(server_cfg)


Found 8 tools:
- get-article
- get-announcements
- get-popular-articles
- get-portals
- make-suggestion
- query-answers
- query-search
- query-retrieve


[Tool(name='get-article', title=None, description='Get Article by ID\n\nGet Article by ID\n\n## Prerequisites\n- Requires a valid portal ID. If you don\'t have the portal ID, first call \'get-portals\' to get available portals.\n- Portal ID format: 2-4 letter prefix + dash + 4-15 digits (e.g., "EB-123456789")\n- Requires a valid article ID. Article ID format: 2-4 letter prefix + dash + 4-15 digits (e.g., "PROD-2996")\n\n## Overview\nThe Get Article by ID API allows a user to retrieve a specific article using its ID. Additional article attributes and contextual views can be specified in the query parameters.\n\nThis API returns structured authoring attributes of Issue, Environment, Cause and Confidence Level when the following conditions are met:\n- The "Allow Structured Authoring" setting is enabled at the partition/department level\n- The "Use Structured Authoring" flag is set on the article type\n', inputSchema={'type': 'object', 'properties': {'request': {'type': 'object', 'properti

## Step 4: Ask in natural language (OpenAI function calling)

Use OpenAI to interpret a prompt and trigger the appropriate MCP tool. Example: “get the popular articles in portal PZ-9999”. Set OPENAI_API_KEY in your environment.


In [12]:
# Requires OPENAI_API_KEY in your environment
# %pip install -U openai

import os, json
from typing import Dict, Any, List

try:
    from openai import OpenAI
except Exception as e:
    raise RuntimeError("Please install OpenAI: pip install openai")

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "")
if not OPENAI_API_KEY:
    raise RuntimeError("Set OPENAI_API_KEY in the first cell or export OPENAI_API_KEY.")

client = OpenAI(api_key=OPENAI_API_KEY)

SYSTEM_PROMPT = (
    "You convert user requests into MCP tool calls. "
    "Choose the best matching function from the provided list and extract arguments from the user's text. "
    "Return exactly one function call with only the required arguments, plus obvious optional ones when specified."
)

async def _discover_openai_functions(server: Dict[str, Any]) -> List[dict]:
    """Discover MCP tools at runtime and convert to OpenAI function schemas."""
    params = StdioServerParameters(
        command=server["command"],
        args=server.get("args", []),
        env=server.get("env", None),
    )
    async with stdio_client(params) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()
            tools_result = await session.list_tools()
            functions: List[dict] = []
            for t in tools_result.tools:
                name = getattr(t, "name", None)
                desc = getattr(t, "description", "") or ""
                schema = getattr(t, "inputSchema", None)
                params_schema = schema if isinstance(schema, dict) and schema.get("type") == "object" else {"type": "object", "properties": {}}
                functions.append({"name": name, "description": desc, "parameters": params_schema})
            return functions

In [13]:
from typing import Dict, Any

async def _call_mcp_direct(server: Dict[str, Any], tool_name: str, arguments: Dict[str, Any]):
    params = StdioServerParameters(
        command=server["command"],
        args=server.get("args", []),
        env=server.get("env", None),
    )
    payload: Dict[str, Any]
    if isinstance(arguments, dict) and "request" in arguments:
        payload = arguments
    else:
        payload = {"request": arguments if isinstance(arguments, dict) else {}}
    async with stdio_client(params) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()
            result = await session.call_tool(tool_name, payload)
            if getattr(result, "content", None):
                item = result.content[0]
                data = getattr(item, "text", None) or getattr(item, "data", None) or str(item)
                return data
            return result

In [ ]:
# Convert MCP JSON output to natural language using OpenAI
# Provides a wrapper that returns a concise, human-readable answer.

from typing import Any


def _stringify_for_llm(obj: Any, max_chars: int = 12000) -> str:
    try:
        s = obj if isinstance(obj, str) else json.dumps(obj, ensure_ascii=False, indent=2)
    except Exception:
        s = str(obj)
    if len(s) > max_chars:
        return s[:max_chars] + "\n...[truncated]..."
    return s


def _mcp_to_natural_language(user_text: str, raw_result: Any) -> str:
    raw_str = _stringify_for_llm(raw_result)
    comp = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": (
                    "Rewrite API/JSON outputs into a concise, clear natural-language answer. "
                    "Do not output JSON. Prefer short paragraphs and bullets when helpful."
                ),
            },
            {
                "role": "user",
                "content": (
                    "User request:\n" + user_text +
                    "\n\nTool output (JSON-like):\n" + raw_str +
                    "\n\nWrite the final answer in natural language for the user."
                ),
            },
        ],
    )
    return comp.choices[0].message.content


async def run_nl_command_natural(server: Dict[str, Any], user_text: str):
    """Natural language → tool selection → MCP call → natural-language answer."""
    resp = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": user_text},
        ],
        functions = await _discover_openai_functions(server_cfg),
        function_call="auto",
    )

    msg = resp.choices[0].message
    if getattr(msg, "function_call", None):
        fn_name = msg.function_call.name
        try:
            fn_args = json.loads(msg.function_call.arguments or "{}")
        except Exception:
            fn_args = {}
        print(f"🔧 LLM chose: {fn_name} with args: {fn_args}")
        result = await _call_mcp_direct(server, fn_name, fn_args)
        print("\n✅ MCP result (raw):\n", result)
        nl = _mcp_to_natural_language(user_text, result)
        print("\n💬 Natural language answer:\n", nl)
        return nl
    else:
        print("❌ LLM did not return a function call. Prompt: ", user_text)
        return None

# Example:
await run_nl_command_natural(server_cfg, "get the popular articles in portal PZ-9999")



🔧 LLM chose: get-popular-articles with args: {'request': {'portalID': 'PZ-9999'}}

✅ MCP result (raw):
 {"ArticleResults":{"articles":[{"articleType":{"articleCategoryId":0,"articleTypeId":"999900000002020","typeName":"General","useStructuredAuthoring":false},"createdBy":{"firstName":"Administrator","id":"1","lastName":"User","userName":"Administrator User"},"createdDate":"2025-10-24T18:01:49.000Z","hasAttachments":false,"id":"PROD-2060","languageCode":"en-US","link":{"href":"/knowledge/portalmgr/v4/portals/PZ-9999/articles/PROD-2060?%24lang=en-US","rel":"self"},"modifiedBy":{"firstName":"Administrator","id":"1","lastName":"User","userName":"Administrator User"},"modifiedDate":"2025-10-24T18:02:05.000Z","name":"A Beginner’s Guide to Machine Learning","versionId":"PROD-1055"},{"articleType":{"articleCategoryId":0,"articleTypeId":"999900000002020","typeName":"General","useStructuredAuthoring":false},"createdBy":{"firstName":"Administrator","id":"1","lastName":"User","userName":"Administr

"Here are some popular articles from portal PZ-9999:\n\n1. [A Beginner’s Guide to Machine Learning](https://yourdomain.com/knowledge/portalmgr/v4/portals/PZ-9999/articles/PROD-2060?%24lang=en-US) - Created on October 24, 2025.\n2. [The Hivemind: Who Are You?](https://yourdomain.com/knowledge/portalmgr/v4/portals/PZ-9999/articles/PROD-2084?%24lang=en-US) - Created on October 24, 2025.\n3. [The Society that Kills Its People: Why South Korea?](https://yourdomain.com/knowledge/portalmgr/v4/portals/PZ-9999/articles/PROD-2074?%24lang=en-US) - Created on October 24, 2025.\n4. [AI and Creativity: Can Machines Be Truly Creative?](https://yourdomain.com/knowledge/portalmgr/v4/portals/PZ-9999/articles/PROD-2061?%24lang=en-US) - Created on October 24, 2025.\n5. [AI and Ethics: Where Do We Draw the Line?](https://yourdomain.com/knowledge/portalmgr/v4/portals/PZ-9999/articles/PROD-2062?%24lang=en-US) - Created on October 24, 2025.\n6. [Can AI Replace Human Jobs? A Realistic Look](https://yourdomain.

In [16]:
# Additional examples ran on the developer's system:

await run_nl_command_natural(server_cfg, "Can you get my portals in the Production department?")
await run_nl_command_natural(server_cfg, "Can you get the chunks about AI and machine learning in pz-9999?")

🔧 LLM chose: get-portals with args: {'request': {'department': 'Production', 'Dollar_lang': 'en-US'}}

✅ MCP result (raw):
 {"AllAccessiblePortals":{"paginationInfo":{"count":3,"pagenum":1,"pagesize":25},"portal":[{"department":{"id":"1001","name":"Production"},"id":"PROD-1001","name":"AI Agent Portal","shortURL":[{"id":"PROD-1008","isDefault":true,"shortURLName":"R98F6q","templateName":"ombre"}]},{"department":{"id":"1001","name":"Production"},"description":"Automatically created portal allowing access to all published articles","id":"PZ-1001","name":"Master Portal","shortURL":[{"id":"PROD-1004","isDefault":true,"shortURLName":"MnIZRi","templateName":"ombre"}]},{"department":{"id":"1001","name":"Production"},"id":"PROD-1000","name":"Testing Portal","shortURL":[{"id":"PROD-1006","isDefault":true,"shortURLName":"hD7BQw","templateName":"ombre"}]}]},"ContentType":"application/json","RawResponse":{},"StatusCode":200}

💬 Natural language answer:
 You have access to three portals in the Prod

"Here are some key insights about AI and machine learning from your request regarding resource PZ-9999:\n\n1. **Understanding Machine Learning**:\n   - **Machine Learning (ML)** is a subset of artificial intelligence that allows systems to learn from experience without explicit programming. \n   - It is categorized into two main types:\n     - **Supervised Learning**: This involves training a model on a labeled dataset to predict outcomes based on input data.\n     - **Unsupervised Learning**: Here, the model identifies patterns and relationships within unlabeled data, useful for tasks like clustering.\n\n2. **Key Machine Learning Techniques**:\n   - **Regression**: This is used to predict continuous outcomes, with types including:\n     - **Linear Regression** for straightforward predictions.\n     - **Polynomial Regression** for complex relationships.\n     - **Logistic Regression** for binary classification tasks.\n   - **Decision Trees**: A graphical model used for classification a